In [0]:
import pyspark
from pyspark.sql import SparkSession
import json
from urllib.request import urlopen
from urllib.error import HTTPError

In [0]:
dbutils.widgets.text('filepath','https://config4.blob.core.windows.net/config/config3.json')

In [0]:
def readingConfigFile(filepath):
    """ This function reads the configuration file from ADLS """
    try:
        file = urlopen(filepath)
    except HTTPError:
        print("Incorrect URL.")
        return
    data = ""
    for line in file: data += line.decode()
    try:
        config = json.loads(data)
        return config
    except ValueError as e:
        return "Decoding JSON file failed"

In [0]:
config = readingConfigFile(dbutils.widgets.get('filepath'))

In [0]:
config

Out[39]: {'key': 'newstorage512',
 'scope': 'testscope',
 'container': 'adventureworks',
 'storageAccountName': 'newstorage512',
 'sourceDir': 'bronzezone',
 'sinkDir': 'silverzone',
 'tablenames': ['Address',
 'Customer',
 'CustomerAddress',
 'Product',
 'ProductCategory',
 'ProductDescription',
 'ProductModel',
 'ProductModelProductDescription',
 'SalesOrderDetail',
 'SalesOrderHeader'],
 'dropColumns': {'Address': ['rowguid'],
 'Customer': ['NameStyle', 'rowguid'],
 'CustomerAddress': ['rowguid'],
 'Product': ['DiscontinuedDate', 'rowguid'],
 'ProductCategory': ['rowguid'],
 'ProductDescription': ['rowguid'],
 'ProductModel': ['rowguid'],
 'ProductModelProductDescription': ['rowguid'],
 'SalesOrderDetail': ['rowguid'],
 'SalesOrderHeader': ['RevisionNumber',
 'Status',
 'OnlineOrderFlag',
 'CreditCardApprovalCode',
 'Comment',
 'rowguid']},
 'nullHandling': {'Address': ['AddressLine2'],
 'Customer': ['Title', 'MiddleName', 'Suffix'],
 'CustomerAddress': [],
 'Product': ['Weight', 'SellEndDate', 'Size', 'Color'],
 'ProductCategory': ['ParentProductCategoryID'],
 'ProductDescription': [],
 'ProductModel': ['CatalogDescription'],
 'ProductModelProductDescription': [],
 'SalesOrderDetail': [],
 'SalesOrderHeader': []}}

In [0]:
spark.conf.set(
    "fs.azure.account.key."+config["storageAccountName"]+".dfs.core.windows.net",
    dbutils.secrets.get(scope=config["scope"],key=config["key"])) 

In [0]:
SalesLT = {}

for table in config['tablenames']:
    try:
        SalesLT[table] = spark.read.format("parquet").option("header","true").load("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/incremental/"+table+"_part.parquet")
    except Exception as e:
        print(e)

In [0]:
for table in config['tablenames']:
    try:
        SalesLT[table].write.format("parquet").option("header","true").mode("append").save("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/landingzone/"+table)
        SalesLT[table].write.format("parquet").option("header","true").mode("append").save("abfss://"+config["container"]+"@"+config["storageAccountName"]+".dfs.core.windows.net/bronzezone/"+table)
    except Exception as e:
        print(e)